### Building and registering the container

The following shell code shows how to build the container image using `docker build` and push the container image to ECR using `docker push`. This code is also available as the shell script `container/build-and-push.sh`, which you can run as `build-and-push.sh sagemaker-auto-categorization` to build the image `sagemaker-auto-categorization`. 

This code looks for an ECR repository in the account you're using and the current default region (if you're using a SageMaker notebook instance, this will be the region where the notebook instance was created). If the repository doesn't exist, the script will create it.

In [47]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-auto-categorization

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/16 : FROM ubuntu:16.04
 ---> 52b10959e8aa
Step 2/16 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> fbb2e117445e
Step 3/16 : WORKDIR /opt
 ---> Using cache
 ---> a00f4eadce34
Step 4/16 : COPY requirements.txt /opt/requirements.txt
 ---> Using cache
 ---> edee025a6091
Step 5/16 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 21aeabbc803e
Step 6/16 : RUN alias python=python3
 ---> Using cache
 ---> 640eadaff4fa
Step 7/16 : RUN wget https://bootstrap.pypa.io/get-pip.py && python3 get-pip.py &&     pip install -r requirements.txt &&         rm -rf /root/.cache
 ---> Using cache
 ---> 90bb757fd509
Step 8/16 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 64b68058ac1b
Step 9/16 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 757ba58cdc6f
Step 10/16 : ENV PATH="/opt/program:${PATH

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Error processing tar file(exit status 1): write /opt/program/learning/trained-models/doc2vec_MM.model: no space left on device


#### In the container the input and output is defined by sagemaker like the following:
###### The input

* `/opt/ml/input/config` contains information to control how your program runs. `hyperparameters.json` is a JSON-formatted dictionary of hyperparameter names to values. These values will always be strings, so you may need to convert them. `resourceConfig.json` is a JSON-formatted file that describes the network layout used for distributed training. Since scikit-learn doesn't support distributed training, we'll ignore it here.
* `/opt/ml/input/data/<channel_name>/` (for File mode) contains the input data for that channel. The channels are created based on the call to CreateTrainingJob but it's generally important that channels match what the algorithm expects. The files for each channel will be copied from S3 to this directory, preserving the tree structure indicated by the S3 key structure. 
* `/opt/ml/input/data/<channel_name>_<epoch_number>` (for Pipe mode) is the pipe for a given epoch. Epochs start at zero and go up by one each time you read them. There is no limit to the number of epochs that you can run, but you must close each pipe before reading the next epoch.

###### The output

* `/opt/ml/model/` is the directory where you write the model that your algorithm generates. Your model can be in any format that you want. It can be a single file or a whole directory tree. SageMaker will package any files in this directory into a compressed tar archive file. This file will be available at the S3 location returned in the `DescribeTrainingJob` result.
* `/opt/ml/output` is a directory where the algorithm can write a file `failure` that describes why the job failed. The contents of this file will be returned in the `FailureReason` field of the `DescribeTrainingJob` result. For jobs that succeed, there is no reason to write this file as it will be ignored.

# Part 2: Training and Hosting your Algorithm in Amazon SageMaker

Once you have your container packaged, you can use it to train and serve models. Let's do that with the algorithm we made above.

## Set up the environment

Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [48]:
# S3 prefix
prefix = 'data/DEMO-auto-categorizer'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [49]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using some articles from CS (AWS RDS). 

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

In [58]:
WORK_DIRECTORY = 'learning/data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [74]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-auto-categorization:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

INFO:sagemaker:Creating training-job with name: sagemaker-auto-categorization-2018-09-05-14-05-22-063


................
Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/gensim/models/doc2vec.py:535: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
Train on 58827 samples, validate on 14707 samples
2018-09-05 14:58:56.806116: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Epoch 1/20
 3904/58827 [>.............................] - ETA: 3:04 - loss: 6.7592 - acc: 0.2072


 7744/58827 [==>...........................] - ETA: 2:32 - loss: 6.6194 - acc: 0.2548


11648/58827 [====>.........................] - ETA: 2:14 - loss: 6.4733 - acc: 0.3019


15552/58827 [======>.......................] - ETA: 2:00 - loss: 6.4018 - acc: 0.3235


...

51744/58827 [=========================>....] - ETA: 18s - loss: 5.5222 - acc: 0.4332


55648/58827 [===========================>..] - ETA: 8s - loss: 5.5216 - acc: 0.4328


58784/58827 [============================>.] - ETA: 0s - loss: 5.5251 - acc: 0.4322


58827/58827 [==============================] - 161s 3ms/step - loss: 5.5248 - acc: 0.4321 - val_loss: 5.5521 - val_acc: 0.4310
Exception during training: [Errno 2] No such file or directory: '/opt/ml/model/lstm-multi-categorizer-larger.model/doc2vec_MM.model.docvecs.vectors_docs.npy'
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/gensim/utils.py", line 679, in save
    _pickle.dump(self, fname_or_handle, protocol=pickle_protocol)
TypeError: file must have a 'write' attribute

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "sagemaker.py", line 16, in <module>
    model.train_and_store_model(input_file, output, new_doc2vec=True)
  File "/opt/program/learning/model.py", line 243, in train_and_store_model
    categorizer.save_model(model_path + '/' + output_file)
  File "/opt/program/learning/model.py", line 161, in save_model
    self.doc2vec.save_model(path + '/doc2vec_MM.model')
  File "/o

ValueError: Error training sagemaker-auto-categorization-2018-09-05-14-05-22-063: Failed Reason: AlgorithmError: Exception during training: [Errno 2] No such file or directory: '/opt/ml/model/lstm-multi-categorizer-larger.model/doc2vec_MM.model.docvecs.vectors_docs.npy'
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/gensim/utils.py", line 679, in save
    _pickle.dump(self, fname_or_handle, protocol=pickle_protocol)
TypeError: file must have a 'write' attribute

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "sagemaker.py", line 16, in <module>
    model.train_and_store_model(input_file, output, new_doc2vec=True)
  File "/opt/program/learning/model.py", line 243, in train_and_store_model
    categorizer.save_model(model_path + '/' + output_file)
  File "/opt/program/learning/model.py", line 161, in save_model
    self.doc2vec.save_model(path + '/doc2vec_MM.model')
  File "/opt/program/lea

## Deploy the model

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [95]:
from sagemaker.predictor import csv_serializer
predictor = tree.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

INFO:sagemaker:Creating model with name: sagemaker-auto-categorization-2018-09-07-07-54-58-317
INFO:sagemaker:Creating endpoint with name sagemaker-auto-categorization-2018-09-05-14-05-22-063


--------------------------------------------------------------!

## Choose some data and use it for a prediction

In order to do some predictions, we'll test the algorithm on some of the data we used for training and do predictions against it. This is, of course, bad statistical practice, but a good way to see how the mechanism works.

## Optional cleanup

When you're done with the endpoint, you'll want to clean it up.

In [94]:
sess.delete_endpoint(predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: sagemaker-auto-categorization-2018-09-05-14-05-22-063
